In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

# check pandas version
print(pd.__version__)
# check numpy version
print(np.__version__)

# read WineKMC spreadsheet data to Pandas dataframe 
winekmc = pd.ExcelFile("WineKMC.xlsx")
# display all the sheets available in WineKMC.xlsx 
print(winekmc.sheet_names)

0.19.2
1.11.3
['OfferInformation', 'Transactions', 'Pivot', 'Matrix', '4MC', '4MC - TopDealsByCluster', 'Distances', '4MC Silhouette', '5MC', '5MC - TopDealsByCluster', '5MC Silhouette', '5MedC', '5MedC - TopDealsByCluster']


In [2]:
# have a peak of the OfferInformation sheet data
df_offerinfo = winekmc.parse("OfferInformation")
print("Total Offer # : %d" % len(df_offerinfo))

# make sure the Offer number column is in sorted in order
df_offerinfo.sort_values("Offer #", axis=0, inplace=True)
display(df_offerinfo.head())

Total Offer # : 32


,Offer #,Campaign,Varietal,Minimum Qty (kg),Discount (%),Origin,Past Peak
0,1,January,Malbec,72,56,France,False
1,2,January,Pinot Noir,72,17,France,False
2,3,February,Espumante,144,32,Oregon,True
3,4,February,Champagne,72,48,France,True
4,5,February,Cabernet Sauvignon,144,44,New Zealand,True


In [3]:
# have a peak of the Transactions sheet data
df_transaction = winekmc.parse("Transactions")
print("Total number of transactions: %d" % len(df_transaction))
display(df_transaction.head())

Total number of transactions: 324


,Customer Last Name,Offer #
0,Smith,2
1,Smith,24
2,Johnson,17
3,Johnson,24
4,Johnson,26


In [4]:
# dont know why aggfunc='count' doesn't work
df_transaction_pivot = pd.pivot_table(df_transaction, index='Offer #', columns='Customer Last Name', aggfunc=len, fill_value=0)
display(df_transaction_pivot.head())
# save the pivot table for later use
cpy_df_transaction_pivot = df_transaction_pivot.copy(deep=True)

# convert pivot table to dataframe
df_transaction_pivot = pd.DataFrame(df_transaction_pivot.to_records())
display(df_transaction_pivot.head())

Customer Last Name,Adams,Allen,Anderson,Bailey,Baker,Barnes,Bell,Bennett,Brooks,Brown,...,Turner,Walker,Ward,Watson,White,Williams,Wilson,Wood,Wright,Young
Offer #,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,Offer #,Adams,Allen,Anderson,Bailey,Baker,Barnes,Bell,Bennett,Brooks,...,Turner,Walker,Ward,Watson,White,Williams,Wilson,Wood,Wright,Young
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# stitch the pivot trasaction table to the Offer Info table
df_offerinfo = df_offerinfo.join(df_transaction_pivot.drop('Offer #', axis=1))
display(df_offerinfo.head())

# confirm same number of records (rows) as before
print("Total Offer # : %d" % len(df_offerinfo))

,Offer #,Campaign,Varietal,Minimum Qty (kg),Discount (%),Origin,Past Peak,Adams,Allen,Anderson,...,Turner,Walker,Ward,Watson,White,Williams,Wilson,Wood,Wright,Young
0,1,January,Malbec,72,56,France,False,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,2,January,Pinot Noir,72,17,France,False,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,February,Espumante,144,32,Oregon,True,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,February,Champagne,72,48,France,True,0,0,0,...,1,0,1,0,0,0,0,0,1,0
4,5,February,Cabernet Sauvignon,144,44,New Zealand,True,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Total Offer # : 32


In [6]:
from sklearn.cluster import KMeans

# so far we made the Offer Info table same as Data Smart Ch02
# but the set-up is different in scikit-learn
# inputs in scikit-learn:
# features: Offer #
# samples: Ppl who bought the beers based on the Offer#

# redo the input table
inputs = cpy_df_transaction_pivot.transpose()
inputs = pd.DataFrame(inputs.to_records())
display(inputs.head())

# to feed the inputs to scikit-learn's KMean, we need to drop the Customer Last Name column
X = inputs.drop('Customer Last Name', axis=1)
display(X.head())

,Customer Last Name,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,Adams,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
1,Allen,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,Anderson,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
3,Bailey,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Baker,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,31,32
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [36]:
# Data Smart Ch02 first set cluster number to 4
k = 4
kmeans = KMeans(n_clusters=k, random_state=42).fit(X)
labels = pd.DataFrame(kmeans.labels_)
labels.columns = ["labels"]

# Now let's check which person belongs to which cluster
result = pd.concat([inputs['Customer Last Name'], labels], axis=1)
result.columns = ['Customer', 'Cluster']
display(result.head())
print("Total Distance %f" % kmeans.inertia_)

,Customer,Cluster
0,Adams,3
1,Allen,0
2,Anderson,2
3,Bailey,3
4,Baker,0


Total Distance 216.954365


In [37]:
# Now let's display the customers in each cluster
df = []
header = []

for l in range(k):
    df.append(pd.DataFrame((result.loc[result['Cluster']==l])['Customer']))
    df[l].reset_index(drop=True, inplace=True)
    header.append("Cluster %d" % l)

cluster_result = pd.concat(df, axis=1)
cluster_result.columns = [header]
display(final_result)

,Cluster 0,Cluster 1,Cluster 2,Cluster 3
0,Allen,Barnes,Anderson,Adams
1,Baker,Brooks,Bell,Bailey
2,Bennett,Butler,Campbell,Brown
3,Cooper,Clark,Cook,Carter
4,Edwards,Davis,Cox,Collins
5,Garcia,Evans,Flores,Cruz
6,Gonzalez,Fisher,Jenkins,Diaz
7,Gray,Foster,Johnson,Hill
8,Green,Gomez,Mitchell,Hughes
9,Gutierrez,Hall,Moore,James


In [53]:
# checking the silhouette score for the 4 cluster solution
from sklearn.metrics import silhouette_score, silhouette_samples

silhouette = silhouette_score(X, np.array(labels["labels"]))
print("Mean Silhouette Coefficient of all samples: %f" % silhouette)

# the mean silhouette score of all samples are close to 0, pretty bad eh, isn't ?
# now let's look at the silhouette score of each sample
silhouettes = silhouette_samples(X, np.array(labels["labels"]))
print (silhouettes)

# None of our samples as a silhouette score > 0.5 !!!
print([s for s in silhouettes if s>0.5])

# Many samples are also put in the wrong cluster according to silhouette score
print("There are %d over %d samples that are put in the wrong cluster" % ( len([s for s in silhouettes if s<0]), len(X)) )

Mean Silhouette Coefficient of all samples: 0.096703
[ 0.30236879  0.04682806  0.41194274  0.18842422  0.08510343 -0.05932899
  0.4579475  -0.03465615 -0.05521847  0.27562272 -0.02291999  0.41143448
  0.2721175  -0.05149472  0.12235792  0.41194274  0.0439342   0.4579475
  0.24582713 -0.07057422  0.27838392  0.08250444 -0.08400892  0.05339914
  0.33855435 -0.01086849  0.03265944 -0.07738535  0.0842914  -0.1058091
  0.04779601  0.01545853 -0.00831872 -0.05887149 -0.00667455  0.22582146
 -0.01035331  0.27616887  0.00484108  0.2502448   0.41194274  0.44698345
  0.10186102  0.07033092  0.14223712 -0.01251696  0.20205291  0.05534387
 -0.06154569 -0.13920515  0.03454864  0.00435924  0.08429896  0.33855435
  0.06920409 -0.03465615  0.44698345  0.05368037  0.2376624   0.09981051
  0.07774339  0.10186102 -0.08172854  0.2376624   0.29157462  0.12461961
  0.33855435  0.03084904 -0.03369012  0.05306987  0.04245384  0.03964042
 -0.06264048 -0.06007367  0.07112583 -0.07321271  0.25123199  0.04466879
